In [1]:
!pip install python-igraph
!pip install cairocffi

     |████████████████████████████████| 3.1 MB 6.7 MB/s 
     |████████████████████████████████| 88 kB 3.5 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.3.0-py3-none-any.whl size=89668 sha256=aee3fe3c309666d4ec5b0cf65242df6b099038b75407fa06251f1b5235804389
  Stored in directory: /root/.cache/pip/wheels/4e/ca/e1/5c8a9692a27f639a07c949044bec943f26c81cd53d3805319f
Successfully built cairocffi


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
gnndatasetpath = "/content/gdrive/Shareddrives/FYP/DataSets/Gnn-Dataset/Test_Datasets"

In [4]:
!mkdir 'figures'

In [5]:
_DEBUG = True

In [20]:
import re
from igraph import *
from collections import defaultdict
import os
import random

In [7]:
#  Bulding the assembly graph

In [22]:
def predict(sequence):
  return random.choice([0,1,2])

In [8]:
def label_sequences(gfafile):
    results = {}
    with open(gfafile) as file:
      line = file.readline()
      while line != "":
          # Identify lines with sequence information
          if "S" in line:
              strings = line.split("\t")
              name, seq = strings[1], strings[2]
              result = predict(seq)
              results[name] = result
          line = file.readline()
    return results

In [30]:
def buld_assembly_graph(cotigpathfile, gfafile, node_count = 0, segment_contigs = {}, paths = {}):

  # cotigpathfile = "/content/dataset-1/contigs.paths"
  # gfafile = "/content/dataset-1/assembly_graph_with_scaffolds.gfa"

  paths = {}
  segment_contigs = {}
  node_colors = {}
  node_count = 0
  node_sequences = defaultdict(set)
  plasmid_colors = []
  chromosome_colors= []

  # Get contig paths from contigs.paths
  class_results = label_sequences(gfafile)
  with open(cotigpathfile) as file:
      name = file.readline()
      path = file.readline()
      
      while name != "" and path != "":
              
          while ";" in path:
              path = path[:-2]+","+file.readline()
          
          start = 'NODE_'
          end = '_length_'
          contig_num = str(int(re.search('%s(.*)%s' % (start, end), name).group(1))-1)
          # print("contig_num:- ", contig_num , "\n")
          
          segments = path.rstrip().split(",")
          
          if contig_num not in paths:
              node_count += 1
              paths[contig_num] = [segments[0], segments[-1]]
              # print("paths[contig_num]:- ", paths[contig_num] , "\n")
          
          class_count = [0, 0, 0]
          for segment in segments:
              if (segment.endswith("+") or segment.endswith("-")):
                org_segment = segment[:-1]
              else:
                org_segment = segment
              
              node_class = class_results[org_segment]
              class_count[node_class] += 1
              
              node_sequences[contig_num].add(segment)
              if segment not in segment_contigs:
                  segment_contigs[segment] = set([contig_num])
              else:
                  segment_contigs[segment].add(contig_num)
          
          # //TODO: Is this correct?
          final_class = sorted(range(len(class_count)), key=lambda i:class_count[i])[-1]
          if (final_class == 0):
            plasmid_colors.append(contig_num)
          elif (final_class == 1):
            chromosome_colors.append(contig_num)
          node_colors[contig_num] = final_class
          print(contig_num, node_colors[contig_num])
          
          name = file.readline()
          path = file.readline()
  links = []
  links_map = defaultdict(set)
  # Get contig paths from contigs.paths
  with open(gfafile) as file:
      line = file.readline()
      
      while line != "":
          
          # Identify lines with link information
          if "L" in line:
              strings = line.split("\t")
              f1, f2 = strings[1]+strings[2], strings[3]+strings[4]
              links_map[f1].add(f2)
              links_map[f2].add(f1)
              links.append(strings[1]+strings[2]+" "+strings[3]+strings[4])
          line = file.readline()
  # Create graph
  g = Graph()
  # Add vertices
  g.add_vertices(node_count)
  for i in range(len(g.vs)):
      g.vs[i]["id"]= i
      g.vs[i]["label"]= str(i+1)
      g.vs[i]["class"] = node_colors[str(i)]
  for i in range(len(paths)):
      segments = paths[str(i)]

      #//TODO: Check
      start = segments[0]
      start_rev = ""
      if start.endswith("+"):
          start_rev = start[:-1]+"-"
      else:
          start_rev = start[:-1]+"+"
          
      end = segments[1]
      end_rev = ""
      if end.endswith("+"):
          end_rev = end[:-1]+"-"
      else:
          end_rev = end[:-1]+"+"
      
      new_links = []
      
      # //TODO Add seperate logic to orientation
      if start in links_map:
          new_links.extend(list(links_map[start]))
      if start_rev in links_map:
          new_links.extend(list(links_map[start_rev]))
      if end in links_map:
          new_links.extend(list(links_map[end]))
      if end_rev in links_map:
          new_links.extend(list(links_map[end_rev]))
      
      # //TODO: Do we need only start and end ?
      for new_link in new_links:
          if new_link in segment_contigs:
              for contig in segment_contigs[new_link]:
                  if i!=int(contig):
                      g.add_edge(i,int(contig))
              
  g.simplify(multiple=True, loops=False, combine_edges=None)
  return g, plasmid_colors, chromosome_colors

In [10]:
# Visualising the Initial Assembly Graph using python-igraph

In [11]:
def visualize_assembly_graph(g, name):
  out_fig_name = "figures/" + name+ ".png"
  visual_style = {}
  # Set bbox and margin
  visual_style["bbox"] = (1500,1500)
  visual_style["margin"] = 30
  # Set vertex colours
  visual_style["vertex_color"] = 'white'
  # Set vertex size
  visual_style["vertex_size"] = 35
  # Set vertex lable size
  visual_style["vertex_label_size"] = 15
  # Don't curve the edges
  visual_style["edge_curved"] = False
  # Set the layout
  my_layout = g.layout_fruchterman_reingold()
  visual_style["layout"] = my_layout
  # Plot the graph
  plot(g, out_fig_name, **visual_style)

In [12]:
# Visualising the Coloured Assembly Graph using python-igraph

In [13]:
def color_assembly_grapg(g, outputname, plasmid_colors, chromosome_colors):
  node_colours = []
  for i in range(node_count):
      if i in plasmid_colors:
          node_colours.append("red")
      elif i in chromosome_colors:
          node_colours.append("green")
      else:
          node_colours.append("white")
  out_fig_name = outputname + "-colored.png"
  g.vs["color"] = node_colours
  visual_style = {}
  # Set bbox and margin
  visual_style["bbox"] = (1500,1500)
  visual_style["margin"] = 30
  # Set vertex size
  visual_style["vertex_size"] = 35
  # Set vertex lable size
  visual_style["vertex_label_size"] = 15
  # Don't curve the edges
  visual_style["edge_curved"] = False
  # Set the layout
  visual_style["layout"] = my_layout
  # Plot the graph
  plot(g, out_fig_name, **visual_style)

In [14]:
def create_disjoint_components(graph):
    components = graph.decompose()
    if(_DEBUG):
      print("# of components", len(components))
    for components in components:
      print(list(components.vs))
    return components

In [18]:
# assign directory
directory = gnndatasetpath
# directory = '/content/dataset-1'

# iterate over files in
# that directory

def process():
  for datafolder in os.listdir(directory):
      dataset = os.path.join(directory, datafolder)
      gfafile = str(dataset) + "/assembly_graph_with_scaffolds.gfa"
      contigpathfile = str(dataset) + "/contigs.paths"
      outputname = datafolder
      graph, plasmid_colors, chromosome_colors = buld_assembly_graph(contigpathfile, gfafile)
      visualize_assembly_graph(graph, outputname)
      color_assembly_grapg(graph, outputname, plasmid_colors, chromosome_colors)
      create_disjoint_components(graph)
      if _DEBUG:
        break


In [31]:
process()

0 0
0 0
1 2
1 2
2 2
2 2
3 1
3 1
4 2
4 2
5 0
5 0
6 2
6 2
7 0
7 0
8 1
8 1
9 0
9 0
10 2
10 2
11 2
11 2
12 0
12 0
13 2
13 2
14 1
14 1
15 2
15 2
16 0
16 0
17 0
17 0
18 2
18 2
19 2
19 2
20 0
20 0
21 2
21 2
22 0
22 0
23 0
23 0
24 2
24 2
25 2
25 2
26 2
26 2
27 1
27 1
28 0
28 0
29 2
29 2
30 2
30 2
31 2
31 2
32 2
32 2
33 2
33 2
34 2
34 2
35 1
35 1
36 0
36 0
37 1
37 1
38 2
38 2
39 1
39 1
40 0
40 0
41 2
41 2
42 1
42 1
43 0
43 0
44 2
44 2
45 2
45 2
46 1
46 1
47 1
47 1
48 2
48 2
49 0
49 0
50 0
50 0
51 1
51 1
52 1
52 1
53 0
53 0
54 2
54 2
55 1
55 1
56 1
56 1
57 2
57 2
58 2
58 2
59 2
59 2
60 1
60 1
61 0
61 0
62 0
62 0
63 1
63 1
64 2
64 2
65 1
65 1
66 0
66 0
67 0
67 0
68 2
68 2
69 2
69 2
70 1
70 1
71 2
71 2
72 2
72 2
73 1
73 1
74 1
74 1
75 1
75 1
76 0
76 0
77 1
77 1
78 0
78 0
79 1
79 1
80 0
80 0
81 2
81 2
82 0
82 0
83 2
83 2
84 1
84 1
85 2
85 2
86 2
86 2
87 0
87 0
88 0
88 0
89 1
89 1
90 0
90 0
91 0
91 0
92 0
92 0
93 2
93 2
94 0
94 0
95 1
95 1
96 2
96 2
97 1
97 1
98 1
98 1
99 0
99 0
100 2
100 2
101 0
10

KeyError: ignored